# Drive Setup

In [ ]:
'''
Google Drive Setup
'''

##google drive set up 
from google.colab import drive
drive.mount('/content/drive')
##moving into required directory 
%ls
%cd drive/
%cd MyDrive/
%cd 'Thesis'/
%ls

# Spotipy Install

In [ ]:
'''
Downloading and installing Spotipy package for scraping spotify
'''

#cloning spotipy from github
#! git clone https://github.com/plamere/spotipy

In [ ]:
# Installing Spotipy
%cd spotipy
!pip3 install spotipy --upgrade

In [ ]:
#Returning to main directory
%cd ..

# Imports

In [ ]:
'''
Importing required packages
'''
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import csv

# Spotify Client setup

In [ ]:
'''
Setting up Spotify Client 
'''
ccm = SpotifyClientCredentials(client_id='ada09f948da24ffc81d748b726429d6d',client_secret='a9f386752fa14b0a86c20e1159ed5eea')
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='ada09f948da24ffc81d748b726429d6d',client_secret='a9f386752fa14b0a86c20e1159ed5eea'))

#Getting Song Data from Spotify Personal

In [ ]:
'''
Getting List of song name
'''
songList = open("SongList/Song List.txt").readlines()

In [ ]:
'''
Scraping Spotify
'''

#List to hold data about each song
spotifyData = []
#Error Counter to keep track of songs were data could not be collectd
error = 0

for song in songList:
  #print(song)
  
  #Making adjustments to song name to change into required query term for use with spotify
  song = song.replace('\n','')
  s = 'track:' + song.split("(")[0]
  #Attempting to get song data 
  try:
    #Using spotify search function to find song from name
    searchDataInit = sp.search(q=s, limit=1, offset=0, type='track', market=None)
    searchData = searchDataInit.get('tracks').get('items')[0]
  
    #Getting artist id
    artistInformation = searchData.get('artists')
    artistID = artistInformation[0].get('id')

    #Getting song release date
    releaseDate = searchData.get('album').get('release_date')

    #Getting song data
    trackLength = searchData.get('duration_ms')
    trackExplicit = searchData.get('explicit')
    trackID = searchData.get('id')
    numAvailableMarkets = len(searchData.get('available_markets'))

    """
    #Print Statements for above variables
 
    print(artistID)
 
    print(releaseDate)
 
    print(trackLength)
    print(trackExplicit)
    print(trackID)
    print(numAvailableMarkets)

    """

    #Getting Audio data for track using track id from above
    trackAudioFeatures = sp.audio_features(tracks=[trackID])
 
    #collecting and storing audio features for track
    trackAcousticness = trackAudioFeatures[0].get('acousticness') 
    trackDanceability = trackAudioFeatures[0].get('danceability')
    trackEnergy = trackAudioFeatures[0].get('energy')
    trackInstrumentalness = trackAudioFeatures[0].get('instrumentalness')
    trackKey = trackAudioFeatures[0].get('key')
    trackLiveness = trackAudioFeatures[0].get('liveness')
    trackLoudness = trackAudioFeatures[0].get('loudness')
    trackMode = trackAudioFeatures[0].get('mode')
    trackSpeechiness = trackAudioFeatures[0].get('speechiness')
    trackTempo = trackAudioFeatures[0].get('tempo')
    trackValence = trackAudioFeatures[0].get('valence')

    """
    #Print block for audio features
 
    print(trackAcousticness)
    print(trackDanceability)
    print(trackEnergy)
    print(trackInstrumentalness)
    print(trackKey)
    print(trackLiveness)
    print(trackLoudness)
    print(trackMode)
    print(trackSpeechiness)
    print(trackTempo)
    print(trackValence)
    """
    #Getting artist data from artist id
    artistData = sp.artist(artist_id=artistID)

    #Getting artistData
    artistFollowers = artistData.get('followers').get('total')
    artistPopularity = artistData.get('popularity')

    """
    #print for artist data
  
    print(artistFollowers)
    print(artistPopularity)
    """
  except:
    #If the above fails 1 is added to the error value and the variables are set to predeterimed levels so that issues can be manually resolved 
    error = error + 1
    trackID = 'NA'
    releaseDate = 'NA'
    trackLength = -9999
    trackExplicit = 'FALSE'
    numAvailableMarkets = -9999
    trackAcousticness = -9999
    trackDanceability = -9999
    trackEnergy = -9999
    trackInstrumentalness = -9999
    trackKey =  -9999
    trackLiveness = -9999
    trackLoudness = -9999
    trackMode = -9999
    trackSpeechiness = -9999
    trackTempo = -9999
    trackValence = -9999
    artistID = 'NA'
    artistFollowers = -9999
    artistPopularity = -9999

  #Creating a List object with the data collected from spotify
  trackData = [song,trackID,releaseDate,trackLength,trackExplicit,numAvailableMarkets,
               trackAcousticness,trackDanceability,trackEnergy,trackInstrumentalness,trackKey,trackLiveness,trackLoudness,trackMode,trackSpeechiness,trackTempo,trackValence,
               artistID,artistFollowers,artistPopularity]
  #Adding the track specific data to the list of all track data
  spotifyData.append(trackData)



In [ ]:
#Print out of the length of the Spotify Data set and Errors to check the number of Errors and make sure no songs from the dataset were missed
print(len(spotifyData))
print(error)

# Write song data to csv

In [ ]:
'''
Writing song data to csv file
'''

#Storing desired names of variables 
fields = ['Song','Track ID', 'Release Date','Track Length','Track Explicit','Number Available Markets',
               'Track Acousticness','Track Danceability','Track Energy','Track Instrumentalness','Track Key','Track Liveness','Track Loudness','Track Mode','Track Speechiness','Track Tempo','Track Valence',
               'Artist ID','Artist Followers','Artist Popularity'] 

# Create new csv to store collected data
f = open('spotifyData.csv', 'w')

# create the csv writer
writer = csv.writer(f)

# writing the fields 
writer.writerow(fields)

# writing the data rows 
writer.writerows(spotifyData)

# close the file
f.close()